#### <b>필요한 라이브러리 설치</b>

In [1]:
# # 트랜스포머 라이브러리 설치(Huggig Face에 있는 모델 불러오기 가능)
# !pip install transformers
# !pip install datasets # 데이터 세트 다운로드 Hugging Face와 연동

#### <b>필요한 라이브러리 불러오기</b>

In [2]:
import copy # 특정한 파이썬 객체를 통째로 메모리에 copy할 때
import json # json 형식으로 데이터를 표현할 때
import logging # 학습 과정 등 전반적인 프로그램의 진행 상황을 로깅할 때
import os # 파일 입출력 등 현재 컴퓨터에 대한 기능 수행할 때

# 경고(warning) 메시지가 너무 많이 나오는 것을 대비하여 무시 처리
import warnings
warnings.filterwarnings("ignore")

# 로깅할 때 기본적으로 오류(error) 사항으로 로그 메시지를 남기겠다는 의미
import logging
logging.basicConfig(level=logging.ERROR)

# 벡터, 행렬 등의 처리를 위한 NumPy, 테이블(엑셀) 형식의 데이터 처리할 때 Pandas
import numpy as np
import pandas as pd

from datasets import load_dataset
# train_test_split: 별도로 구분된 validation 세트가 없을 때
# 학습 데이터 세트에서 일부를 train과 validation으로 나눌 때 자주 사용 (8:2 정도로 나눔)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import torch

In [3]:
import torch

# GPU 사용 가능 여부 확인
print("CUDA Available:", torch.cuda.is_available())

if torch.cuda.is_available():
    # 현재 활성화된 GPU 인덱스 확인
    print("Current GPU Index:", torch.cuda.current_device())
    
    # 현재 활성화된 GPU의 이름 확인
    print("Current GPU Name:", torch.cuda.get_device_name(torch.cuda.current_device()))
    
    # GPU 메모리 사용량 확인
    print("GPU Memory Allocated:", torch.cuda.memory_allocated())
    print("GPU Memory Cached:", torch.cuda.memory_cached())

CUDA Available: True
Current GPU Index: 0
Current GPU Name: GeForce RTX 2080 Ti
GPU Memory Allocated: 0
GPU Memory Cached: 0


In [4]:
df = pd.read_csv('10000_labeled.csv')
df = df.fillna(0)

In [5]:
df["선호도"].unique()

array([3, 1, 2, 4, 0])

In [6]:
df["비속어"].unique()

array([1., 0.])

In [7]:
df["정치"].unique()

array([0., 1.])

#### <b>학습할 모델 관련 라이브러리 불러오기</b>

* <b>Hugging Face Auto Class</b>
  * <b>Auto 키워드</b>: 사전 학습된(pre-trained) 모델을 이용해 원하는 작업을 수행한다.
    * → 최대한 자동화할 수 있게 만들어진 라이브러리
    * <b>AutoTokenizer</b>: 입력을 토큰(token)으로 바꾸는 기능을 수행한다.
사전 학습된 tokenizer를 불러올 수 있다.
    * <b>AutoModelForSequenceClassification</b>: 문장 분류(classification)을 위한 자동화된 모델을 제공한다.
      * forward() 함수 자체가 사전에 설정한 num_classes와 일치
      * 만약 우리가 multi-task learning을 한다면, 이걸 그대로 사용하기는 어렵다.
    * 만약 multi-task learning을 하는 상황에서는 대신에 <b>AutoModel</b>을 이용해서 직접 수정하는 것이 효과적
    * <b>AutoModel</b>: 꼭 텍스트 분류 목적이 아니라, BERT 등의 모델을 불러올 수 있도록
  * 사전 학습된 BERT 모델을 불러올 수 있다.
    * 예시) bert-base-multilingual-sentiment
  * from_pretrained() 함수를 이용해 특정한 경로에서 모델을 불러올 수 있다.
  * PyTorch 혹은 TensorFlow 상관없이 사용할 수 있다.

In [8]:
import transformers
# Auto Model For Sequence Classification: 텍스트 분류를 위한 모델 → Cross-Entropy loss 사용
from transformers import AutoConfig, AutoModel, AutoModelForSequenceClassification, AutoTokenizer
# linear_schedule_with_warmup: 단계적으로 learning rate 줄여나가는 방법
# AdamW: SGD와 같이 optimization 방법 중 하나
from transformers import AdamW, get_linear_schedule_with_warmup

#### <b>본 실습에서 어떤 모델을 쓸 것인가?</b>

* KoBigBird를 사용하고, 다음과 같은 형태로 사용 가능
  * KoBigBird: BigBird 특유의 sparse attention 사용 (default)

<pre>
from transformers import AutoModel, AutoTokenizer

# by default its in `block_sparse` mode with num_random_blocks=3, block_size=64
model = AutoModel.from_pretrained("monologg/kobigbird-bert-base")

# you can change `attention_type` to full attention like this:
model = AutoModel.from_pretrained("monologg/kobigbird-bert-base", attention_type="original_full")

# you can change `block_size` & `num_random_blocks` like this:
model = AutoModel.from_pretrained("monologg/kobigbird-bert-base", block_size=16, num_random_blocks=2)

tokenizer = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base")
text = "한국어 BigBird 모델을 공개합니다!"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
</pre>

In [9]:
from transformers import AutoModel, AutoTokenizer

# by default its in `block_sparse` mode with num_random_blocks=3, block_size=64
# 이름에서부터 알 수 있듯이 KoBigBird는 BERT 기반의 모델
model = AutoModel.from_pretrained("monologg/kobigbird-bert-base")

# Tokenizer도 마찬가지로 BERT 기반에서 가져온 것을 확인
tokenizer = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base")
text = "한국어 BigBird 모델을 공개합니다!"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

Attention type 'block_sparse' is not possible if sequence_length: 12 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


In [10]:
# KoBigBird에 들어가는 입력 형태를 보니까 다음과 같은 형식을 지키면 된다.
# {"input_ids": 값, "token_type_ids": 값, "attention_mask": 값}
print(encoded_input)

# input_ids: [[CLS], x_1, x_2, ..., x_9, x_10 [SEP]]
# token_type_ids: 전부 [MASK] 토큰이 아니므로, 0의 값을 채워 넣은 것으로 보임 (지금 inference 하는 상황이므로 당연)
# attention_mask: 학습할 때 cheating 방지하려고 0을 가끔 채워 넣음 (지금은 inference 하는 상황이므로 1이 당연)

{'input_ids': tensor([[    2, 11802, 32418,  4616,  9101,  4565,  7498,  4604,  7220, 15763,
           505,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [11]:
# output의 형태는 "BaseModelOutputWithPoolingAndCrossAttentions" 객체
# 그 중 첫 번째 속성은 last_hidden_state
# 형태는 다음과 같음 → [batch_size (1개 문장이니까), seq_len (토큰의 길이), hidden_size (임베딩 차원)]
# → BERT와 정확히 동일
print(output.last_hidden_state.shape)

# [[CLS], x_1, x_2, ..., x_9, x_10 [SEP]] => 총 길이가 12개
# [ 768차원, 768차원, ..., 768차원]
# 보통 첫 번째 토큰인 [CLS] 토큰의 임베딩 값인 768차원의 벡터만 사용

# 각 데이터마다 [CLS] 토큰만 가져오기
print(output.last_hidden_state[:,0,:].shape)

torch.Size([1, 12, 768])
torch.Size([1, 768])


In [12]:
# 기초 설정상 last_hidden_state와 pooler_output만 출력
print(output.last_hidden_state[0][0][:10]) # 마지막 레이어 hidden_states에서 [CLS] 토큰 임베딩
# pre-training task에 사용된 결과
print(output.pooler_output[0][:10]) # 마지막 레이어 hidden_states에서 [CLS] 토큰 임베딩에 추가적인 처리를 거친 결과

tensor([ 0.0744,  0.1033, -0.4811,  0.0936, -0.2172, -0.2245, -0.3635, -0.5025,
         0.3230,  0.1951], grad_fn=<SliceBackward0>)
tensor([ 0.4389, -0.4116, -0.1959, -0.0865, -0.2311,  0.3522, -0.1873, -0.1011,
         0.3908, -0.4379], grad_fn=<SliceBackward0>)


#### <b>실험을 위한 하이퍼 파라미터 설정</b>

In [13]:
# SimpleNamespace는 온점(.)으로 속성의 값을 정의할 수 있도록 해주는 라이브러리
# config.task = "cls"라고 하면, 나중에 print(config.task)했을 때 "cls"라고 출력
from types import SimpleNamespace

config = SimpleNamespace()

config.task = "cls"
config.dataset = "comment"

config.cache_dir = "cache" # 현재 데이터 세트에 대한 단어집 등 현재 task를 위한 임시적인 폴더
config.output_dir = "output" # 최종적인 모델이 저장되고, 결과가 저장되는 폴더

config.use_tpu = False
config.model_name_or_path = "monologg/kobigbird-bert-base" # Model name or path (HuggingFace에서 불러와 사용할 모델 이름)
config.data_dir = "./" # The input data dir ("10000_labeled.csv"가 있는 경로)

# 실질적으로 학습을 위해서는 tokenizing이 수행된 train file과 predict file을 만들어야 한다.
config.train_file = "10000_labeled.csv" # 미리 준비된 학습 데이터 세트 경로
# 어차피 10000_labeled.csv를 (1) training 목적, (2) validatoin 목적으로 쪼개니까 평가는 validation으로 결과가 나올 것임
config.predict_file = "10000_labeled.csv" # 미리 준비된 평가 데이터 세트 경로
# → 이거 일단 없으면, 지금처럼 train_file로 넣으시되, 나중에 생기시면 갈아끼우기

config.max_seq_length = 1024 # The maximum total input sequence length after tokenization. (최대 토큰 길이)
config.train_batch_size = 4 # Batch size for training. (학습할 때 batch_size)
config.eval_batch_size = 2 # Batch size for evaluation. (평가할 때 batch_size)

config.learning_rate = 3e-5 # The initial learning rate for Adam. (Adam optimizer에서 쓸 learning rate)
config.num_train_epochs = 10 # Total number of training epochs to perform. (전체 학습 epoch 수)

config.preference_classes = 5
config.slang_classes = 1
config.politic_classes = 1
config.gradient_accumulation_steps = 2 # Number of updates steps to accumulate before performing a backward/update pass.
# batch_size가 큰 것처럼 처리하기 위해서, backward()를 매 번 수행하지 않고, gradient를 누적(acculmulation)하는 것

config.threads = 4
config.seed = 42 # random seed for initialization

config.do_train = True # Whether to run training.
config.do_eval_during_train = True
config.do_eval = True # Whether to run prediction.

config.do_lower_case = False
config.weight_decay = 0.0 # Weight decay if we apply some.
config.adam_epsilon = 1e-8 # Epsilon for Adam optimizer.
config.max_grad_norm = 1.0 # Max gradient norm.
config.warmup_proportion = 0.0 # Warmup proportion for linear warmup
# BigBird에서는 full attention을 하면, 메모리는 조금 더 소모되지만, 더 정확도가 향상
# config.attention_type = "original_full"

#### <b>학습 데이터 전처리</b>

* 학습 텍스트를 매번 tokenizing을 하지 않고, <b>모델 학습 시작 전에 미리 모든 텍스트를 tokenizing</b> 한 결과를 저장한다.
  * 이렇게 했을 때, 결과적인 속도는 훨씬 빠르기 때문이다.

In [14]:
# 본 실습에서 사용할 tokenizer 객체 초기화
tokenizer = AutoTokenizer.from_pretrained(config.model_name_or_path, cache_dir=config.cache_dir)

In [51]:
def train_split(config, texts, labels, is_train):
    # 지금 평가 결과는 validation에 대한 결과
    # [오류] stratify가 labels면, test_dataset에 특정 레이블이 아예 등장하지 않으면 오류 발생
    """
    x_train, y_train, x_label, y_label = train_test_split(
        texts, labels, test_size=0.2, random_state=config.seed, stratify=labels
    )
    """
    x_train, y_train, x_label, y_label = train_test_split(
        texts, labels, test_size=0.2, random_state=config.seed, stratify=None
    )
    if is_train:
        texts, labels = x_train, x_label
    else:
        texts, labels = y_train, y_label
    return texts, labels

# 댓글(comment)이 담긴 .csv 파일이 있을 때, 여기에서 텍스트와 레이블 추출
def process_comment_cls(config, data_file, is_train):
    df = pd.read_csv(data_file)
    df = df.fillna(0)  # nan값 0으로 채우기
    df = df[df["선호도"]!= -1] # 선호도 -1 인 문장 학습에서 배제
    df = df[:500]
    # 매 줄에서 "선호도", "비속어", "comment" 열 추출
    preferences = df["선호도"].astype(int).values.tolist()
    slangs = df["비속어"].astype(int).values.tolist()
    politics = df["정치"].astype(int).values.tolist()
    labels = []
    # 한 줄씩 데이터를 확인하며
    for i in range(len(preferences)):
        preference = preferences[i] # 선호도
        slang = slangs[i] # 비속어
        politic = politics[i] # 정치
        labels.append([preference, slang, politic])
    texts = df["comment"].astype(str).values.tolist()
    texts, labels = train_split(config, texts, labels, is_train)
    return texts, labels

In [16]:
df = pd.read_csv("10000_labeled.csv")
df = df.fillna(0)
df = df[df["선호도"]!= -1]
df = df[4000:5000]
df["comment"].astype(str).values.tolist()

['독일가서도 폭격기라 한녀 안먹을거같은데ㅋㅋㅋ',
 '털카스새끼들 발작하는거보니까 개꿀잼이네\\',
 '15슼 라인업은 전설이다',
 '똥남아 뒷골목선에서 정리',
 '한녀는 빚쌓여서 성매매해도 나라 지원인데 한남은 빚쌓이면 얄짤없이 땡이노ㅠㅠ',
 '러시아 남성의 평균 수명은 61.8세고 대한민국 남성 평균수명은 80.6세다.',
 '회장님 곧 시멘트 낭낭하게 채워지시겠노 ㄷㄷㄷㄷ',
 '기안도 회장앞에선 절절기면서 제대로 그려주네 ㅋㅋㅋ 이게 롱런 비결이지 ㅋㅋ',
 '저게 맞지ㅋㅋ 롤 6년 해도 만년 골드고 하지 다 그런거지 뭐',
 '2등 누나가 너무 이쁜데',
 '쳐맞을때 불쌍햇음',
 '저런 캣맘년들한테 고양이 입양하면 절대안됨  하루에 최소 몇백번씩 고양이사진찍어보내라고하고 1주일에 5번 가정방문한다고하고 교통비,식비 다내놓으라고하더라 ㅅㅂ년들 저게 정신병자지 사람새끼냐',
 '이거 왜맛있냐 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ',
 '사람 뇌 이식 됐나보네',
 'ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ ㄹㅇ 멍청한게 벼슬',
 '이강 물딸배 라고해야돼나',
 '한녀야 제발 혼자 살다 뒤져  앵겨붙지 말고',
 '최진실 딸 쟤 말은 좀 걸러서 들어야함전적이 화려함거의 양치기소년급임',
 '에휴 카메라하고 뭔 상관 관점의 문제인데',
 '참나 제작진 맘대로 개입하고 난리부르스칠꺼면 일반인왜 섭외하냐?그 통통튀는맛에 일반인 섭외하는건데 즤들이 다 짜맞출거면?? 어이없네 ㅋㅋㅋ',
 '알아서 뒤져서 국민연금 기초연금 부담 덜어주겠다는데 희생정신에 고마워하자',
 '근데 왜 까는거임?',
 '개꼴리게생겼네 rule34 딱대라 씹련 ㅋㅋ',
 '미국=흑인들을 위한 나라 미국=흑인들을 위한 나라 미국=흑인들을 위한 나라 미국=흑인들을 위한 나라 미국=흑인들을 위한 나라 미국=흑인들을 위한 나라 미국=흑인들을 위한 나라 미국=흑인들을 위한 나라 미국=흑인들을 위한 나라 미국=흑인들을 위한 나라 미국=흑인들을 위한 나라 미국=흑인들을 위한 나라 미국=흑인들을 위한

In [64]:
import torch.utils.data as torch_data

def data_pretokenizing(config, tokenizer, is_train=True):
    if is_train:
        data_file = config.train_file
    else:
        data_file = config.predict_file

    data_path = config.data_dir
    if data_file is not None:
        data_path = os.path.join(data_path, data_file)
    else:
        data_path += "/"

    # 실제로 tokenizer를  저장될 데이터 세트의 파일 이름이 바로 dataset_file
    comps = [
        data_path,
        config.dataset,
        config.model_name_or_path.replace("/", "_"),
        config.max_seq_length,
        "train" if is_train else "dev",
        "dataset.txt",
    ]
    dataset_file = "_".join([str(comp) for comp in comps])
    print("dataset_file:", dataset_file)

    # 기존에 write_samples() 함수가 하는 역할: 텍스트 문장을 읽어와서 token 값만 저장
    # 멀티 쓰레드 방식으로 구현되어 있어서, 더 (비효율적이지만) 이해하기 쉬운 코드로 돌려 놓음
    with open(dataset_file, "w", encoding="utf-8") as writer_file:
        # data: "10000_labeled.csv" 파일에서 읽어와 (텍스트, 선호도 레이블)만 남긴 .csv 파일
        cnt = 0
        total_data = process_comment_cls(config, data_path, is_train)
        # 학습 데이터 세트를 하나씩 확인하며
        for text, label in zip(total_data[0], total_data[1]):
            # 여기에서 data는 하나의 (텍스트, 레이블) 쌍
            # feature는 해당 텍스트를 tokenizer에 넣어서 나온 결과
            feature = tokenizer(text, max_length=config.max_seq_length, padding="max_length", truncation=True, add_special_tokens=True)
            # 실제로 학습을 위해서는 (각 토큰의 index, 정답 레이블)로 학습을 진행
            writed_data = {
                "input_ids": feature["input_ids"],
                "attention_mask": feature["attention_mask"],
                "preference": int(float(label[0])), 
                "slang": int(float(label[1])),
                "politic": int(float(label[2])), 
            }
            #JSON은 쉽게 말하면 Python에서 dictionary와 같음 → 이를 file로 저장하는 것
            writer_file.write(json.dumps(writed_data) + "\n")
            cnt += 1
        print(f"{cnt} features processed from {data_path}")

    return dataset_file

In [65]:
# 본 코드에서 학습을 수행하려는 경우
if config.do_train:
    # 학습 데이터 세트 전처리
    train_dataset_file = data_pretokenizing(config, tokenizer=tokenizer)

# 평가 데이터 세트 전처리(validation = dev 같은 의미)
predict_dataset_file = data_pretokenizing(config, tokenizer=tokenizer, is_train=False)

# 결과적으로 만들어진 "./10000_labeled.csv_comment_monologg_kobigbird-bert-base_1024_train_dataset.txt"
# 내용을 확인해 보면, 약 8,000개의 각 학습 데이터에 대하여
#   → 하나씩 {"input_ids", "attention_mask", "preference", "slang"}으로 구성

dataset_file: ./10000_labeled.csv_comment_monologg_kobigbird-bert-base_1024_train_dataset.txt
400 features processed from ./10000_labeled.csv
dataset_file: ./10000_labeled.csv_comment_monologg_kobigbird-bert-base_1024_dev_dataset.txt
100 features processed from ./10000_labeled.csv


#### <b>데이터 로더 초기화</b>

In [66]:
class IterableDatasetPad(torch.utils.data.IterableDataset):
    def __init__(
        self,
        dataset: torch.utils.data.IterableDataset,
        batch_size: int = 1,
        num_devices: int = 1,
        seed: int = 0,
    ):
        self.dataset = dataset
        self.batch_size = batch_size
        self.seed = seed
        self.num_examples = 0

        chunk_size = self.batch_size * num_devices
        length = len(dataset)
        self.length = length + (chunk_size - length % chunk_size)

    def __len__(self):
        return self.length

    def __iter__(self):
        self.num_examples = 0
        if (
            not hasattr(self.dataset, "set_epoch")
            and hasattr(self.dataset, "generator")
            and isinstance(self.dataset.generator, torch.Generator)
        ):
            self.dataset.generator.manual_seed(self.seed + self.epoch)

        first_batch = None
        current_batch = []
        for element in self.dataset:
            self.num_examples += 1
            current_batch.append(element)
            # Wait to have a full batch before yielding elements.
            if len(current_batch) == self.batch_size:
                for batch in current_batch:
                    yield batch
                    if first_batch is None:
                        first_batch = batch.copy()
                current_batch = []

        while self.num_examples < self.length:
            add_num = self.batch_size - len(current_batch)
            self.num_examples += add_num
            current_batch += [first_batch] * add_num
            for batch in current_batch:
                yield batch
            current_batch = []

In [67]:
# 전처리된 데이터는 하나하나 {"input_ids", "attention_mask", "labels", ...} 형태를 가짐
# PyTorch가 하나의 배치를 처리할 때는 PyTorch Tensor 형태여야 함
# <데이터 로더에서 불러오는 "Tensor"를 정의하는 함수>
def collate_fn(features):
    input_ids = [sample["input_ids"] for sample in features]
    attention_mask = [sample["attention_mask"] for sample in features]
    
    preference = [sample["preference"] for sample in features]
    slang = [sample["slang"] for sample in features]
    politic = [sample["politic"] for sample in features]

    input_ids = torch.tensor(np.array(input_ids).astype(np.int64), dtype=torch.long)
    attention_mask = torch.tensor(np.array(attention_mask).astype(np.int8), dtype=torch.long)
    
    preference = torch.tensor(np.array(preference).astype(np.int64), dtype=torch.long)
    slang = torch.tensor(np.array(slang).astype(np.int64), dtype=torch.float)
    politic = torch.tensor(np.array(politic).astype(np.int64), dtype=torch.float)
    inputs = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
    }
    labels = {
        "preference": preference,
        "slang": slang,
        "politic": politic
    }
    return inputs, labels

# 본 코드에서 학습을 수행하려는 경우
if config.do_train:
    # 학습 데이터 로더 초기화
    train_dataset = load_dataset("text", data_files=train_dataset_file, download_mode="force_redownload")["train"]
    train_dataset = train_dataset.map(lambda x: json.loads(x["text"]), batched=False)

    train_dataloader = torch_data.DataLoader(
        train_dataset,
        sampler=torch_data.RandomSampler(train_dataset),
        drop_last=False,
        batch_size=config.train_batch_size,
        collate_fn=(collate_fn),
    )

# 평가 데이터 세트 전처리(validation = dev 같은 의미)
predict_dataset = load_dataset("text", data_files=predict_dataset_file, download_mode="force_redownload")["train"]
predict_dataset = predict_dataset.map(lambda x: json.loads(x["text"]), batched=False)
predict_dataset = IterableDatasetPad(
    dataset=predict_dataset,
    batch_size=config.eval_batch_size,
    num_devices=1,
    seed=config.seed,
)

predict_dataloader = torch_data.DataLoader(
    predict_dataset,
    sampler=None,
    drop_last=False,
    batch_size=config.eval_batch_size,
    collate_fn=(collate_fn),
)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

#### <b>텍스트 분류 모델 정의</b>

In [68]:
from transformers import AutoModel
import torch.nn as nn

# 텍스트 분류 모델 정의
class ClsModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # (전처리된) 학습할 데이터 세트의 경로 설정
        data_file = os.path.join(config.data_dir, str(config.train_file))
        # 사전 학습된 모델 가중치 불러오기
        model_config = AutoConfig.from_pretrained(config.model_name_or_path, num_labels=5)
        self.model = AutoModel.from_pretrained(
            config.model_name_or_path, config=model_config, cache_dir=config.cache_dir
        )
        self.classifier1 = nn.Linear(768, 5) # 선호도 분류기
        self.classifier2 = nn.Linear(768, 1) # 비속어 분류기
        self.classifier3 = nn.Linear(768, 1) # 정치 편향 분류기

        # BERT model과 별개로, 입력 문자열을 토큰의 index로 바꾸어주는 tokenizer가 사용됨
        self.tokenizer = tokenizer

    # "학습된 모델"을 save_dir에 저장하는 함수
    def save_pretrained(self, save_dir):
        self.model.save_pretrained(save_dir)
        # Tokenizer는 기본적으로 "special_tokens_map_file", "tokenizer_file"을 가질 수 있음
        # 이러한 값을 제거한 뒤에 save_dir에 저장하겠다는 의미
        for key in ["special_tokens_map_file", "tokenizer_file"]:
            self.tokenizer.init_kwargs.pop(key, None)
        self.tokenizer.save_pretrained(save_dir)

    def get_optimizer(self): # 현재 모델을 학습하기 위한 최적화 방법(AdamW) 객체를 불러오는 함수
        # bias랑 LayerNorm에는 decay 적용하지 않겠다는 의미
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in self.model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": config.weight_decay,
            },
            {
                "params": [p for n, p in self.model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": config.weight_decay,
            },
        ]
        # AdamW의 첫 번째 인자(params)는 "학습할 가중치", weight_decay는 가중치에 적용되는 regularization 기법
        optimizer = AdamW(optimizer_grouped_parameters, lr=config.learning_rate, eps=config.adam_epsilon)
        return optimizer

    def get_scheduler(self, batch_num, optimizer): # AdamW로 학습할 때, learning rate을 단계적으로 줄이기 위한 함수
        if config.warmup_proportion == 0.0:
            return None

        t_total = batch_num // config.gradient_accumulation_steps * config.num_train_epochs

        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=int(t_total * config.warmup_proportion),
            num_training_steps=t_total,
        )

        return scheduler

    def forward(self, inputs):
        # print(inputs) => {"input_ids", "attention_mask"}
        # https://huggingface.co/docs/transformers/model_doc/big_bird
        # BigBirdModel의 입력 양식에 맞게 넣어주어야 오류가 없음
        # BigBirdModel로 "input_ids", "attention_mask" 등 "미리 정해진" 규격에 맞는 입력만
        # 들어가야 오류가 없다는 의미 => 그러므로, preference, slang 등은 들어가면 X
        hidden = self.model(**inputs)
        # 마지막 레이어의 [CLS] 토큰만 가져오기
        cls_token_embeddings = hidden.last_hidden_state[:,0,:] # [batch_size, 768]
        output_1 = self.classifier1(cls_token_embeddings) # 768 → 5
        output_2 = torch.sigmoid(self.classifier2(cls_token_embeddings)) # 768 → 1
        output_3 = torch.sigmoid(self.classifier3(cls_token_embeddings)) # 768 → 1

        return output_1, output_2, output_3
    

    def eval_step(self, inputs, labels, outputs):
        logits_1 = outputs[0].detach().cpu()
        logits_2 = outputs[1].detach().cpu()
        logits_3 = outputs[2].detach().cpu()
        labels_1 = self.tensor_to_list(labels["preference"])
        labels_2 = self.tensor_to_list(labels["slang"])
        labels_3 = self.tensor_to_list(labels["politic"])
        threshold = 0.5
        predictions_1 = self.tensor_to_list(torch.argmax(logits_1, dim=-1))
        predictions_2 = self.tensor_to_list(logits_2.squeeze(1) >= threshold)
        predictions_3 = self.tensor_to_list(logits_3.squeeze(1) >= threshold)
        results_1 = [{"prediction": prediction, "label": label} for prediction, label in zip(predictions_1, labels_1)]
        results_2 = [{"prediction": prediction, "label": label} for prediction, label in zip(predictions_2, labels_2)]
        results_3 = [{"prediction": prediction, "label": label} for prediction, label in zip(predictions_3, labels_3)]
        
        return {"results_1": results_1, "results_2": results_2, "results_3": results_3}

    # PyTorch의 Tensor 객체를 NumPy 객체로 변환
    def tensor_to_array(self, tensor):
        return tensor.detach().cpu().numpy()

    # PyTorch의 Tensor 객체를 Python의 리스트(list) 자료형으로 변환
    def tensor_to_list(self, tensor):
        return self.tensor_to_array(tensor).tolist()

In [69]:
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

def cal_running_avg_loss(loss, running_avg_loss, decay=0.99):
    if running_avg_loss == 0:
        return loss
    running_avg_loss = running_avg_loss * decay + (1 - decay) * loss
    return running_avg_loss

#### <b>모델 학습 및 평가 라이브러리</b>

In [70]:
from functools import partial
import sklearn.metrics as sklearn_metrics

# for binary-classification
binary_metrics = {
    "accuracy": sklearn_metrics.accuracy_score,
    "precision": sklearn_metrics.precision_score, # TP / (TP + FP)
    "recall": sklearn_metrics.recall_score, # recall = sensitivity (민감도)
    "f1": sklearn_metrics.f1_score,
    "matthews_corrcoef": sklearn_metrics.matthews_corrcoef,
    "roc_auc": sklearn_metrics.roc_auc_score,
}

# for multi-classification
metrics = {
    "accuracy": sklearn_metrics.accuracy_score,
    "f1-macro": partial(sklearn_metrics.f1_score, average="macro"),
}

def eval_cls(results_1, results_2, results_3, **kwargs):
    
    print(f"result1: {results_1}")    
    
    predictions_1 = np.array([result["prediction"] for result in results_1])
    labels_1 = np.array([result["label"] for result in results_1])
    predictions_2 = np.array([result["prediction"] for result in results_2])
    labels_2 = np.array([result["label"] for result in results_2])
    predictions_3 = np.array([result["prediction"] for result in results_3])
    labels_3 = np.array([result["label"] for result in results_3])  

    results_1 = {
        metric: round(f(labels_1, predictions_1) * 100, 2)
        for metric, f in metrics.items()
    }
    results_2 = {
        metric: round(f(labels_2, predictions_2) * 100, 2)
        for metric, f in binary_metrics.items()
    }
    results_3 = {
        metric: round(f(labels_3, predictions_3) * 100, 2)
        for metric, f in binary_metrics.items()
    }

    return {
        "results_1": results_1,
        "results_2": results_2,
        "results_3": results_3,
        "best_score_1": results_1["f1-macro"],
        "best_score_2": results_2["f1"],
        "best_score_3": results_3["f1"],
    }

In [71]:
# _run_epoch() 함수는, 우리가 흔히 알고있는 학습 함수와 동일
def _run_epoch(model, loader, device=None, context=None, **kwargs):
    config = kwargs["config"]
    is_train = kwargs["is_train"]

    avg_loss = 0
    results = []
    batch_num = len(loader)

    if is_train:
        model.train()
        if config.use_tpu:
            optimizer = context.getattr_or(
                "optimizer",
                lambda: model.get_optimizer(),
            )
            scheduler = context.getattr_or(
                "scheduler",
                lambda: model.get_scheduler(batch_num, optimizer),
            )
        else:
            optimizer = kwargs["optimizer"]
            scheduler = kwargs["scheduler"]
    else:
        model.eval()

    is_master = True

    pbar = tqdm(enumerate(loader), total=batch_num, disable=not is_master, dynamic_ncols=True, position=0, leave=True)

    corrected_1 = 0
    corrected_2 = 0
    corrected_3 = 0
    total = 0
    
    for i, (inputs, labels) in pbar:
        # inputs: {"input_ids": [batch_size(4), seq_len, 768], "attention_mask": [batch_size(4), seq_len, 768]}
        # labels: {"preference": [batch_size(4), 1], "slang": [batch_size(4), 1], "politic": [batch_size(4), 1]}
        if not config.use_tpu:
            # (k, v) => ("input_ids", value)
            # (k, v) => ("attention_mask", value)
            for k, v in inputs.items():
                if isinstance(v, torch.Tensor):
                    inputs[k] = v.to(device)
            for k, v in labels.items():
                if isinstance(v, torch.Tensor):
                    labels[k] = v.to(device)

        outputs = model(inputs)

        outputs_1 = outputs[0]
        outputs_2 = outputs[1]
        outputs_3 = outputs[2]

        # 첫 번째 정답 레이블은 preference 정답 [batch_size] => 스칼라 (cuda)
        labels_1 = labels["preference"]
        # 두 번째 정답 레이블은 slang 정답 [batch_size] => 스칼라 (cuda)
        labels_2 = labels["slang"]
        # 세 번째 정답 레이블은 politic 정답 [batch_size] => 스칼라 (cuda)
        labels_3 = labels["politic"]

        # Cross-Entropy는 기본적으로 [batch_size, 클래수 개수]랑 [batch_size, 정답 레이블 int 자료형]
        loss_function_1 = nn.CrossEntropyLoss()
        loss_1 = loss_function_1(outputs_1, labels_1) # 말 그대로 scalr (cuda)

        total += outputs_1.shape[0] # batch_size만큼 더해주기
        _, predicted1 = outputs_1.max(1)
        corrected_1 += predicted1.eq(labels_1).sum().item() 

        # BCE Loss는 기본적으로 [batch_size(sigmoid 거친 결과)]랑 [batch_size]를 비교
        loss_function_2 = nn.BCELoss()
        loss_2 = loss_function_2(outputs_2.squeeze(1), labels_2)  # 말 그대로 scalr (cuda)
        threshold = 0.5
        predicted2 = outputs_2.squeeze(1) >= threshold
        corrected_2 += predicted2.eq(labels_2).sum().item()

        loss_function_3 = nn.BCELoss()
        loss_3 = loss_function_3(outputs_3.squeeze(1), labels_3)
        threshold = 0.5
        predicted3 = outputs_3.squeeze(1) >= threshold
        corrected_3 += predicted3.eq(labels_3).sum().item()

        #print(f"선호도 예측 학습 정확도: {corrected_1 / total}, 비속어 예측 학습 정확도: {corrected_2 / total}, 정치 예측 학습 정확도: {corrected_3 / total}")
        w_1 = 1
        w_2 = 1
        w_3 = 1
        loss = w_1 * loss_1 + w_2 * loss_2 + w_3 * loss_3

        # AutoClass는 loss가 내부적으로 사용됨 => cross entropy 쓰임.
        avg_loss = cal_running_avg_loss(loss.item(), avg_loss)
        loss /= config.gradient_accumulation_steps

        if is_train:
            loss.backward()
            if i % config.gradient_accumulation_steps == 0 or i == batch_num - 1:
                if config.max_grad_norm > 0:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), config.max_grad_norm)

                optimizer.step()
                optimizer.zero_grad()

                if scheduler is not None:
                    scheduler.step()
        
        # 평가 모드일 때 호출되는 부분
        else:
            result = (model.module if hasattr(model, "module") else model).eval_step(inputs, labels, outputs)
            results.extend(result)

        if is_master:
            pbar.set_description(
                f"epoch: {kwargs['epoch'] + 1}, {('train' if is_train else 'valid')} loss: {min(100, round(avg_loss, 4))}"
            )
    print(f"_run_epoch results: {results}")
    # 현재 epoch에 대해서 학습이 끝나고 나면 (loss, result)를 반환
    return {
        "loss": avg_loss,
        "result": results,
    }

# 학습 코드에서 호출하는 함수
def run_epoch(**kwargs):
    model = kwargs.pop("model")
    if kwargs["config"].use_tpu:
        results = model(_run_epoch, **kwargs)
    else:
        results = _run_epoch(model, **kwargs)
   
    if isinstance(results, list):
        loss = sum([result["loss"] for result in results]) / len(results)
        result = []
        for res in results:
            result.extend(res["result"])
        results = {"loss": loss, "result": result}

    return results

#### <b>딥러닝 모델 초기화 및 설정</b>

In [72]:
# 현재 모델 이름이 "monologg/kobigbird-bert-base" 이므로, Hugging Face에서 찾아서 불러옴
set_seed(config.seed)

model = ClsModel()  # 딥러닝 모델 초기화

print(f"configuration: {str(config)}")

if torch.cuda.is_available():
    gpu_count = torch.cuda.device_count()
    print(f"{gpu_count} GPU device detected")
    devices = ["cuda:{}".format(i) for i in range(gpu_count)]
    model_dp = torch.nn.DataParallel(model, device_ids=devices)
    #model_dp.to(devices[0])  #GPU 병렬처리
    model.to(devices[0])
else: 
    devices = ["cpu"]
    model_dp = model

# 학습 결과를 저장하기 위한 폴더 만들기
if not os.path.exists(config.cache_dir):
    os.makedirs(config.cache_dir)

output_dir = os.path.join(config.output_dir, config.task, config.dataset)
print("Output directory:", output_dir)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 실제 학습을 위해 optimizer 및 scheduler 초기화
optimizer = None
scheduler = None
if config.do_train: # 학습 모드(train mode)인 경우
    optimizer = model.get_optimizer()
    scheduler = model.get_scheduler(len(train_dataloader), optimizer)

params = {
    "config": config,
    "model": model_dp,
    "optimizer": optimizer,
    "scheduler": scheduler,
}
if not config.use_tpu:
    params["device"] = devices[0]

configuration: namespace(task='cls', dataset='comment', cache_dir='cache', output_dir='output', use_tpu=False, model_name_or_path='monologg/kobigbird-bert-base', data_dir='./', train_file='10000_labeled.csv', predict_file='10000_labeled.csv', max_seq_length=1024, train_batch_size=4, eval_batch_size=2, learning_rate=3e-05, num_train_epochs=10, preference_classes=5, slang_classes=1, politic_classes=1, gradient_accumulation_steps=2, threads=4, seed=42, do_train=True, do_eval_during_train=True, do_eval=True, do_lower_case=False, weight_decay=0.0, adam_epsilon=1e-08, max_grad_norm=1.0, warmup_proportion=0.0)
4 GPU device detected
Output directory: output/cls/comment


#### 모델 학습하기

In [73]:
def do_eval(epoch):
    with torch.no_grad():
        results = run_epoch(loader=predict_dataloader, epoch=epoch, is_train=False, **params)["result"]
        print(results)
        results_1 = [inner_list[0] for inner_list in results]
        results_2 = [inner_list[1] for inner_list in results]
        results_3 = [inner_list[2] for inner_list in results]

        eval_results = eval_cls(
            config=config,
            model=model,
            loader=predict_dataloader,
            tokenizer=model.tokenizer,
            results_1=results_1,
            results_2=results_2,
            results_3=results_3,
        )
        
        print("Eval results for Preference.")
        for k, v in eval_results["results_1"].items():
            print(f"{k} : {v}")

        print("Eval results for Slang.")
        for k, v in eval_results["results_2"].items():
            print(f"{k} : {v}")

        print("Eval results for Politic.")
        for k, v in eval_results["results_3"].items():
            print(f"{k} : {v}")

    return eval_results["best_score_1"], eval_results["best_score_2"], eval_results["best_score_3"]


train_losses = []
val_accuracies = []
if config.do_train:
    best_score = (0, 0, 0)
    for epoch in range(config.num_train_epochs):
        train_results = run_epoch(loader=train_dataloader, epoch=epoch, is_train=True, **params)
        train_loss = train_results['loss']
        train_losses.append(train_loss)

        if config.do_eval_during_train:
            score1, score2, score3 = do_eval(epoch)
            val_accuracies.append((score1, score2, score3))

            if score1 >= best_score[0] and score2 >= best_score[1] and score3 >= best_score[2]:
                best_score = (score1, score2, score3)
                output_dir = os.path.join(config.output_dir, config.task, config.dataset, f"{epoch}-{best_score[0]}-{best_score[1]}-{best_score[2]}-ckpt")
                copy.deepcopy(
                    model_dp.module
                    if hasattr(model_dp, "module")
                    else model_dp._models[0]
                    if hasattr(model_dp, "_models")
                    else model_dp
                ).cpu().save_pretrained(output_dir)
                with open(os.path.join(output_dir, "finetune_config.json"), "w") as save_config:
                    json.dump(vars(config), save_config, sort_keys=True, indent=4)
                print(f"Checkpoint {output_dir} saved.")

epoch: 1, train loss: 2.4632: 100%|██████████████████████████████████████| 100/100 [01:56<00:00,  1.16s/it]


_run_epoch results: []


epoch: 1, valid loss: 1.6797: 100%|████████████████████████████████████████| 51/51 [00:08<00:00,  6.37it/s]


_run_epoch results: ['results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'results_1', 'results_2', 'results_3', 'res

TypeError: string indices must be integers

In [ ]:
import matplotlib.pyplot as plt

# 학습 손실 그래프
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label="Train Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.legend()
plt.show()

# 검증 정확도 그래프
if config.do_eval_during_train:
    val_scores = list(zip(*val_accuracies))
    plt.figure(figsize=(10, 5))
    plt.plot(val_scores[0], label="Preference Accuracy")
    plt.plot(val_scores[1], label="Slang Accuracy")
    plt.plot(val_scores[2], label="Politic Accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.title("Validation Accuracy")
    plt.legend()
    plt.show()

#### Test Dataset Inference

In [ ]:
def inference(model, tokenizer, sentences, device):
    inference_result_1 = []
    inference_result_2 = []
    inference_result_3 = []

    model.eval()
    with torch.no_grad():
        for sentence in sentences:
            inputs = tokenizer.encode_plus(
                sentence, return_tensors="pt", padding="max_length", truncation=True, max_length=config.max_seq_length
            )
            inputs = {key: value.to(device) for key, value in inputs.items()}  # 입력을 GPU로 이동
            outputs = model(inputs)  # 모델 추론 수행
            outputs_1 = outputs[0]
            outputs_2 = outputs[1]
            outputs_3 = outputs[2]
            predictions_1 = torch.argmax(outputs_1.logits, dim=-1)  # 예측된 클래스 인덱스 추출
            predictions_2 = torch.argmax(outputs_2.logits, dim=-1)
            predictions_3 = torch.argmax(outputs_3.logits, dim=-1)
            inference_result_1.append(predictions_1.item())  # 결과 리스트에 추가
            inference_result_2.append(predictions_2.item())
            inference_result_3.append(predictions_3.item())

    return inference_result_1, inference_result_2, inference_result_3

In [ ]:
test_df = pd.read_csv("test_comment.csv")
sentences_to_infer = test_df["comment"].tolist()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inference_result_1, inference_result_2, inference_result_3 = inference(model, model.tokenizer, sentences_to_infer, device)

In [ ]:
test_df["선호도"] = inference_result_1
test_df["비속어"] = inference_result_2
test_df["정치"] = inference_result_3

In [ ]:
test_df

In [ ]:
output_csv_path = "inference_results.csv"
test_df.to_csv(output_csv_path, index=False)